In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
from pprint import pprint
from IPython.core.display import display, HTML
from scipy.stats.mstats import gmean 
import argparse, logging, tempfile, json, sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)
plt.rcParams['figure.max_open_warning'] = 100
pd.options.mode.chained_assignment = None

display(HTML("<style>.container { width:100% !important; }</style>"))
from trace_utils import *

In [46]:
%%capture
trace_file = "./libgpumon_activities_425511.json"
trim_trace(trace_file, 0.90, 1.0)
base_trace_dir = run_ATC()
print(base_trace_dir)
with open(base_trace_dir + "/two_iteration_trace.json") as two:
    two_iteration_stats = json.load(two)

ops = []
roots, cc, corrected_start_time, corrected_end_time = process_event_hierarchy(two_iteration_stats, skip_module=False)
get_operators(roots, ops)

In [54]:
# Sizes of C*B and C'*A
def get_addmm_backward_size(op):
    sizes = []
    for x in op.children:
        if x.name() == "mm":
            size = x.input_shape()
            sizes.append((size[0][0], size[0][1], size[1][1]))
    return sizes

# Never seen BmmBackward0 having only one bmm. Possible though.
def get_bmm_backward_size(op):
    sizes = []
    for x in op.children:
        if x.name() == "bmm":
            size = x.input_shape()
            sizes.append((size[0][0], size[0][1], size[0][2], size[1][2]))
    return sizes

# Not working in new traces as the output_nr op calls are removed
def get_embedding_lookup_forward_size(op):
    sizes = []
    rows_per_block = None
    for x in op.children:
        if x.name() == "output_nr":
            sizes.append(x.input_shape())
        if x.name() == "cudaLaunchKernel":
            ex = cc[op.external_id()]["callees"][x.correlation_id()]["executor"]
            rows_per_block = ex.event["args"]["block"][1]
    D = int(sizes[0][0][1])
    T = int(sizes[1][0][0])
    E = int(sizes[0][0][0] / T)
    B = int((sizes[3][0][0] - 1) / T)
    L = int(sizes[2][0][0] / B / T)
    return B, E, T, L, D, rows_per_block

# Not working in new traces as the size op calls are removed
def get_embedding_lookup_backward_size(op):
    sizes = []
    rows_per_block = -1
    for x in op.children:
        if x.name() == "size":
            sizes.append(x.input_shape())
        if x.name() == "cudaLaunchKernel":
            ex = cc[op.external_id()]["callees"][x.correlation_id()]["executor"]
            rows_per_block = ex.event["args"]["block"][1]
    T = int(sizes[0][0][0])
    E = int(sizes[1][0][0] / T)
    D = int(sizes[1][0][1])
    B = int((sizes[2][0][0] - 1) / T)
    return B, E, T, _, D, rows_per_block

In [15]:
fc_stats = pd.read_csv("./fully_connected_forward.csv", delimiter=',')
fc_stats = preprocessing(fc_stats)
fc_stats = fc_stats[fc_stats["kernel_name"].str.startswith("volta")].reset_index(drop=True)

### Performance models

In [16]:
L2_size = 6 * 1024 * 1024 * 4
num_SM = 80
peak_dram_bw = 809 # GB/s
peak_l2_bw = 2888 # GB/s
peak_throughput = 12200 # GFLOPS

In [87]:
def embedding_forward_predictor(peak_dram_bw, peak_l2_bw, L2_size, num_SM, **kwargs):
    # hit_rate = C(X, L) / C(E, L), X = avg_num_rows_per_table
    def hit_rate(X, E, L):
        ret = 1.0
        e = E
        x = X
        for idx in range(L):
            ret *= x / e
            x -= 1
            e -= 1
        return ret

    # Average number of rows per table in L2
    y = kwargs
    num_total_warps = y["batch_size"] * y["num_tables"] # Total warp number of the kernel
    num_warps_per_sm = y["rows_per_block"] # Number of warps per sm
    num_warps_simul = num_SM * num_warps_per_sm # Total number of warps simultaneously running on the device
    num_tables_simul = (num_warps_simul + y["batch_size"] - 1) // y["batch_size"] # Number of tables simultaneously being accessed on the device
    avg_table_size = min(L2_size // num_tables_simul, y["num_embeddings"] * y["embedding_dim"] * 4) # Average table size that reside on the device
    indices_size = 0 if y["shmem"] else div_round_up(y["bag_size"] * 4, 32) * 32
    avg_num_rows_per_table = (avg_table_size - indices_size) // 4 // y["embedding_dim"]

    # Hit rate
    hr = hit_rate(avg_num_rows_per_table, y["num_embeddings"], y["batch_size"])
    
    # num_thread_x
    num_thread_x = max(y["embedding_dim"] / 4, 1024 / y["rows_per_block"])

    # Traffics
    table_offsets_traffic = 32
    offsets_traffic = 32
    if y["shmem"]:
        indices_dram_traffic = div_round_up(y["bag_size"] * 4, 32) * 32
        indices_l2_traffic = 0
    else: # no_shmem
        indices_dram_traffic = div_round_up(y["bag_size"] * 4, 32) * 32
        indices_l2_traffic = y["embedding_dim"] // (4 * num_thread_x) * div_round_up(y["bag_size"] * 4, 32) * 32
    table_traffic = y["bag_size"] * (div_round_up(y["embedding_dim"] * 4, 32) * 32)
    output_traffic = (div_round_up(y["embedding_dim"] * 4, 32) * 32)

    # avg_table_size all as dram traffic
    # 21, 26, 13, 7, 4, 4, 24, (0.2 ± 0.21)
    total_l2_traffic = ((table_offsets_traffic + offsets_traffic + indices_l2_traffic) * y["batch_size"] + \
                        hr * (table_traffic * y["batch_size"] - avg_table_size)) * y["num_tables"]
    total_dram_traffic = ((indices_dram_traffic + output_traffic) * y["batch_size"] + \
                          (1 - hr) * (table_traffic * y["batch_size"] - avg_table_size) + avg_table_size) * y["num_tables"]

    return max(total_dram_traffic / peak_dram_bw / 1000.0, total_l2_traffic / peak_l2_bw / 1000.0)
        
# e.g. 4340 vs 4789
embedding_forward_predictor(peak_dram_bw, peak_l2_bw, L2_size, num_SM, batch_size=4096, num_embeddings=500000, num_tables=197, bag_size=32, embedding_dim=32, rows_per_block=128, shmem=True)

4340.7676440049445

In [75]:
def embedding_backward_sgd_predictor(peak_dram_bw, **kwargs):
    y = kwargs
    if y["shmem"]: # 40% GMAE...
        indices_traffic = div_round_up(y["bag_size"] * 4, 32) * 32
        grad_output_traffic = div_round_up(y["embedding_dim"] * 4, 32) * 32
    else: # backward_sgd_no_shmem
        indices_traffic = y["bag_size"] * 32
        grad_output_traffic = (y["bag_size"] * div_round_up(y["embedding_dim"] * 4, 32) * 32) * 2

    # Traffic per warp = t_offsets + t_table_offsets + t_indices + t_weights + t_grad_outputs
    total_traffic_per_warp = 32 + \
                            32 + \
                            indices_traffic + \
                            2 * y["bag_size"] * (div_round_up(y["embedding_dim"] * 4, 32) * 32) + \
                            grad_output_traffic

    # Traffic = warp * traffic per warp
    total_traffic = y["batch_size"] * y["num_tables"] * total_traffic_per_warp

    # Total compute throughput
    mac_per_warp = y["bag_size"] * 4 * (y["embedding_dim"] // 4)
    total_mac = y["batch_size"] * y["num_tables"] * mac_per_warp

    return max(total_traffic / peak_dram_bw / 1000, total_mac / peak_throughput / 1000)

# e.g 86291 vs 77508
embedding_backward_sgd_predictor(peak_dram_bw, batch_size=2048, num_embeddings=200000, num_tables=128, bag_size=128, embedding_dim=128, rows_per_block=32, shmem=False)

86291.71294932015

In [77]:
def embedding_backward_rowwise_adagrad_approx_predictor(peak_dram_bw, **kwargs):
    y = kwargs
    
    # Traffic = warp * traffic per warp
    total_traffic_per_warp = 32 + \
                            32 + \
                            2 * (div_round_up(y["bag_size"] * 4, 32) * 32) + \
                            y["bag_size"] * (div_round_up(y["embedding_dim"] * 4, 32) * 32) + \
                            (y["bag_size"] + 1) * (div_round_up(y["embedding_dim"] * 4, 32) * 32) + \
                            y["bag_size"] * (div_round_up(y["embedding_dim"] * 4, 32) * 32)
    total_traffic = y["batch_size"] * y["num_tables"] * total_traffic_per_warp

    return total_traffic / peak_dram_bw / 1000.0

# e.g 64226 vs 63304
embedding_backward_rowwise_adagrad_approx_predictor(peak_dram_bw, batch_size=2048, num_embeddings=200000, num_tables=128, bag_size=128, embedding_dim=128, rows_per_block=32)

64226.252103831896

In [83]:
total_time = 0.0
for op in ops:
    t = 0.0
    if op.name() == "addmm":
        size = op.input_shape()
        M, K, N = size[1][0], size[1][1], size[2][1]
        t = fc_forward_predictor(peak_dram_bw, peak_throughput, fc_stats, batch_size=1, M=M, N=N, K=K)
#         print("addmm", M, N, K, t)
    if op.name() == "bmm":
        size = op.input_shape()
        batch_size, M, K, N = size[0][0], size[0][1], size[0][2], size[1][2]
        t = fc_forward_predictor(peak_dram_bw, peak_throughput, fc_stats, batch_size=batch_size, M=M, N=N, K=K)
#         print("bmm", batch_size, M, N, K, t)
    if op.name() == "LookupFunction":
        B, E, T, L, D, rows_per_block = get_embedding_lookup_forward_size(op)
        lks = []
        for c in op.children:
            if c.name() == "cudaLaunchKernel":
                lks.append(c)
        callees = cc[lks[0].external_id()]["callees"]
        shmem = list(callees.values())[0]["executor"].name().split(',')[1].strip()
        t = embedding_forward_predictor(peak_dram_bw, peak_l2_bw, L2_size, num_SM, batch_size=B, num_embeddings=E, num_tables=T, bag_size=L, embedding_dim=D, rows_per_block=rows_per_block, shmem=shmem)
#         print("Embedding forward", t)
    if op.name() == "LookupFunctionBackward":
        B, E, T, _, D, rows_per_block = get_embedding_lookup_backward_size(op)
        L = 38 # TODO: Cannot get it from trace. Hard code it.
        sgd = False
        lks = []
        for c in op.children:
            if c.name() == "cudaLaunchKernel":
                lks.append(c)
        callees = cc[lks[0].external_id()]["callees"]
        kernel_name = list(callees.values())[0]["executor"].name()
        if "sgd" in kernel_name:
            sgd = True
        shmem = kernel_name.split(',')[1].strip()
        if sgd:
            t = embedding_backward_sgd_predictor(peak_dram_bw, batch_size=B, num_embeddings=E, num_tables=T, bag_size=L, embedding_dim=D, rows_per_block=rows_per_block, shmem=shmem)
        else:
            t = embedding_backward_rowwise_adagrad_approx_predictor(peak_dram_bw, batch_size=B, num_embeddings=E, num_tables=T, bag_size=L, embedding_dim=D, rows_per_block=rows_per_block)
#         print("Embedding backward", t)
    if op.name() == "AddmmBackward":
        sizes = get_addmm_backward_size(op)
        for size in sizes:
            M, K, N = size
            t += fc_forward_predictor(peak_dram_bw, peak_throughput, fc_stats, batch_size=1, M=M, N=N, K=K)
#         print("AddmmBackward", t)
    if op.name() == "BmmBackward0":
        sizes = get_bmm_backward_size(op)
        for size in sizes:
            batch_size, M, K, N = size
            t += fc_forward_predictor(peak_dram_bw, peak_throughput, fc_stats, batch_size=batch_size, M=M, N=N, K=K)
#         print("BmmBackward0", t)
    total_time += t
print("total_time:", total_time)

total_time: 12117.71546046202


In [11]:
# def fc_forward_predictor(peak_dram_bw, peak_throughput, df, **kwargs):
#     def get_record(df, **kwargs):
#         row_count = df.shape[0]
#         condition = pd.Series([True] * row_count)
#         for k, v in kwargs.items():
#             condition = condition & (df[k] == v)
#         return df[condition]

#     def get_closest(df, **kwargs):
#         no_match = {}
#         row_count = df.shape[0]
#         condition = pd.Series([True] * row_count)
#         for k, v in kwargs.items():
#             if v in df[k].unique():
#                 condition = condition & (df[k] == v)
#             else:
#                 no_match[k] = v

#         # With matched dimensions
#         data_points = [(df[condition], {})]

#         # For each of the non-matched dimension
#         for k, v in no_match.items():
#             tmp = []
#             for dp, limits in data_points:
#                 uni_val = sorted(dp[k].unique())

#                 low, high = -1, -1
#                 if v < uni_val[0]:
#                     high = uni_val[0]
#                 elif v > uni_val[-1]:
#                     low = uni_val[-1]
#                 else:
#                     for idx in range(len(uni_val[:-1])):
#                         if uni_val[idx] < v and uni_val[idx+1] > v:
#                             high = uni_val[idx+1]
#                             low = uni_val[idx]
#                             break
#                 assert not (low == -1 and high == -1)

#                 less_tmp = dp[dp[k] == (low if low != -1 else uni_val[0])]
#                 more_tmp = dp[dp[k] == (high if high != -1 else uni_val[-1])]
#                 if low == -1:
#                     less_tmp[k] = 0
#                 if high == -1:
#                     more_tmp[k] = sys.maxsize # Big enough for BW in GB/s or throughput in GFLOPS
#                 tmp_limits = limits.copy()
#                 tmp_limits[k] = (low, high)
#                 tmp.append((less_tmp, tmp_limits))
#                 tmp.append((more_tmp, tmp_limits))
#             data_points = tmp

#         return data_points

#     #####################
#     #       |     X |
#     #    |==O=======|
#     #    |  |       |
#     #    |  |-------O-
#     #    |     X     |
#     #    O           |
#     #    |===========O
#     #    |  X        |
#     #####################

#     record = get_record(df, **kwargs)
#     if not record.empty:
#         return record["kernel_runtime"].iloc[0]
#     data_points = get_closest(df, **kwargs)

#     effective_flops = 0.0
#     effective_bw = 0.0
#     batch_size = kwargs["batch_size"]
#     M = kwargs["M"]
#     N = kwargs["N"]
#     K = kwargs["K"]
#     for dp, limits in data_points:
#         dp_flops_contrib = 0.0
#         dp_bw_contrib = 0.0

#         # An idea, if zero occurs, it's always the bottleneck. A zero dominates all peaks.
#         zero_exists = False
#         peak_exists = False
#         for k, v in limits.items():
#             metric = dp[k].iloc[0]
#             if metric == 0:
#                 zero_exists = True
#             elif metric == sys.maxsize:
#                 peak_exists = True

#         for k, v in limits.items():
#             low, high = v
#             if high == -1: # Reaching the peak, taking average
#                 ratio_l, ratio_h = 0.5, 0.5
#             elif low == -1: # Reaching the bottom, set low as 0
#                 ratio_l, ratio_h = kwargs[k] / high, (high - kwargs[k]) / high
#             else: # Normal, weighted
#                 ratio_l, ratio_h = (kwargs[k] - low) / (high - low), (high - kwargs[k]) / (high - low)
#             # Edge cases: when more than one metric is MAX/0

#             metric = dp[k].iloc[0]
#             if zero_exists:
#                 throughput = 0
#                 dram_bw = 0
#                 ratio = 0
#             elif peak_exists:
#                 throughput = peak_throughput
#                 dram_bw = peak_dram_bw
#                 ratio = ratio_h
#             elif metric == low:
#                 throughput = (dp["batch_size"] * dp["M"] * dp["N"] * dp["K"]).iloc[0] / dp["kernel_runtime"].iloc[0] / 1000 # GFLOPS
#                 dram_bw = (dp["batch_size"] * (dp["M"] * dp["K"] + dp["K"] * dp["N"] + dp["M"] * dp["N"])).iloc[0] / dp["kernel_runtime"].iloc[0] / 1000 * 4 # GB/s
#                 ratio = ratio_l 
#             elif metric == high:
#                 throughput = (dp["batch_size"] * dp["M"] * dp["N"] * dp["K"]).iloc[0] / dp["kernel_runtime"].iloc[0] / 1000 # GFLOPS
#                 dram_bw = (dp["batch_size"] * (dp["M"] * dp["K"] + dp["K"] * dp["N"] + dp["M"] * dp["N"])).iloc[0] / dp["kernel_runtime"].iloc[0] / 1000 * 4 # GB/s
#                 ratio = ratio_h

#             dp_flops_contrib += throughput * ratio
#             dp_bw_contrib += dram_bw * ratio

#         effective_flops += dp_flops_contrib / len(limits.items())
#         effective_bw += dp_bw_contrib / len(limits.items())

#     effective_flops /= len(data_points) / 2
#     effective_bw /= len(data_points) / 2

#     FLOP = kwargs["batch_size"] * kwargs["M"] * kwargs["N"] * kwargs["K"]
#     DRAM_bytes = kwargs["batch_size"] * (kwargs["M"] * kwargs["K"] + kwargs["K"] * kwargs["N"] + kwargs["M"] * kwargs["N"]) * 4
#     predicted_runtime = max(FLOP / effective_flops, DRAM_bytes / effective_bw) / 1000

#     return predicted_runtime

# # 5829 vs 7548
# fc_forward_predictor(peak_dram_bw, peak_throughput, fc_stats, batch_size=256, M=512, N=1000, K=400)

5829.673816702252